In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  # es para enseñar todas las columnas del df
pd.set_option('display.max_rows', None)     # es para enseñar todas las filas del df


import warnings
warnings.simplefilter('ignore')   # es para quitar warnings

## ACTOR

In [10]:
df_actor=pd.read_csv('../data/actor.csv')
df_actor.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [3]:
df_actor.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   actor_id     200 non-null    int64 
 1   first_name   200 non-null    object
 2   last_name    200 non-null    object
 3   last_update  200 non-null    object
dtypes: int64(1), object(3)
memory usage: 41.0 KB


In [9]:
len(df_actor.drop_duplicates())==len(df_actor)  # no hay duplicados

True

## CATEGORY

In [11]:
df_category=pd.read_csv('../data/category.csv')
df_category.head()

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [12]:
df_category.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category_id  16 non-null     int64 
 1   name         16 non-null     object
 2   last_update  16 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4 KB


In [26]:
len(df_category.drop_duplicates())==len(df_category)  # no hay duplicados

True

## FILM

In [17]:
df_film=pd.read_csv('../data/film.csv')
df_film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [14]:
df_film.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1000 non-null   int64  
 1   title                 1000 non-null   object 
 2   description           1000 non-null   object 
 3   release_year          1000 non-null   int64  
 4   language_id           1000 non-null   int64  
 5   original_language_id  0 non-null      float64
 6   rental_duration       1000 non-null   int64  
 7   rental_rate           1000 non-null   float64
 8   length                1000 non-null   int64  
 9   replacement_cost      1000 non-null   float64
 10  rating                1000 non-null   object 
 11  special_features      1000 non-null   object 
 12  last_update           1000 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 495.3 KB


In [27]:
len(df_film.drop_duplicates())==len(df_film)  # no hay duplicados

True

## INVENTORY

In [34]:
df_inventory=pd.read_csv('../data/inventory.csv')
df_inventory.head()

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


In [22]:
df_inventory.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   inventory_id  1000 non-null   int64 
 1   film_id       1000 non-null   int64 
 2   store_id      1000 non-null   int64 
 3   last_update   1000 non-null   object
dtypes: int64(3), object(1)
memory usage: 97.8 KB


In [28]:
len(df_inventory.drop_duplicates())==len(df_inventory)  # no hay duplicados

True

## LANGUAGE

In [7]:
df_language=pd.read_csv('../data/language.csv')
df_language.head()

,language_id,name,last_update
0,1,English,2006-02-15 05:02:19
1,2,Italian,2006-02-15 05:02:19
2,3,Japanese,2006-02-15 05:02:19
3,4,Mandarin,2006-02-15 05:02:19
4,5,French,2006-02-15 05:02:19


In [30]:
df_inventory.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   inventory_id  1000 non-null   int64 
 1   film_id       1000 non-null   int64 
 2   store_id      1000 non-null   int64 
 3   last_update   1000 non-null   object
dtypes: int64(3), object(1)
memory usage: 97.8 KB


In [32]:
len(df_inventory.drop_duplicates())==len(df_inventory)  # no hay duplicados

True

## RENTAL

In [8]:
df_rental=pd.read_csv('../data/rental.csv')
df_rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [31]:
df_rental.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rental_id     1000 non-null   int64 
 1   rental_date   1000 non-null   object
 2   inventory_id  1000 non-null   int64 
 3   customer_id   1000 non-null   int64 
 4   return_date   1000 non-null   object
 5   staff_id      1000 non-null   int64 
 6   last_update   1000 non-null   object
dtypes: int64(4), object(3)
memory usage: 254.0 KB


In [33]:
len(df_inventory.drop_duplicates())==len(df_inventory)  # no hay duplicados

True

### old_HDD - Relaciones titulo, actor y categoría

In [54]:
df_old_HDD=pd.read_csv('../data/old_HDD.csv')
df_old_HDD.head()

,first_name,last_name,title,release_year,category_id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14


In [55]:
df_relacion.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   first_name    1000 non-null   object
 1   last_name     1000 non-null   object
 2   title         1000 non-null   object
 3   release_year  1000 non-null   int64 
 4   category_id   1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 207.6 KB


In [56]:
len(df_old_HDD.drop_duplicates())==len(df_old_HDD)  # no hay duplicados

True

In [57]:
df_old_HDD[['title', 'category_id']][df_old_HDD.title == 'CHEAPER CLYDE']

,title,category_id
4,CHEAPER CLYDE,14
492,CHEAPER CLYDE,14


In [59]:
(df_old_HDD['first_name'] + ' ' + df_old_HDD['last_name']).unique()

array(['PENELOPE GUINESS', 'NICK WAHLBERG', 'ED CHASE', 'JENNIFER DAVIS',
       'JOHNNY LOLLOBRIGIDA', 'BETTE NICHOLSON', 'GRACE MOSTEL',
       'MATTHEW JOHANSSON', 'JOE SWANK', 'CHRISTIAN GABLE', 'ZERO CAGE',
       'KARL BERRY', 'UMA WOOD', 'VIVIEN BERGEN', 'CUBA OLIVIER',
       'FRED COSTNER', 'HELEN VOIGHT', 'DAN TORN', 'BOB FAWCETT',
       'LUCILLE TRACY', 'KIRSTEN PALTROW', 'ELVIS MARX', 'SANDRA KILMER',
       'CAMERON STREEP', 'KEVIN BLOOM', 'RIP CRAWFORD', 'JULIA MCQUEEN',
       'WOODY HOFFMAN', 'ALEC WAYNE', 'SANDRA PECK', 'SISSY SOBIESKI',
       'TIM HACKMAN', 'MILLA PECK', 'AUDREY OLIVIER', 'JUDY DEAN',
       'BURT DUKAKIS', 'VAL BOLGER', 'TOM MCKELLEN', 'GOLDIE BRODY'],
      dtype=object)

In [63]:
# Hago un left join, haciendo que en films y old_HDD el index sea title.


joined = df_film.set_index('title').join(df_old_HDD[["title", "category_id"]].set_index('title'), lsuffix='', rsuffix='_dcha', how='left')


<class 'pandas.core.frame.DataFrame'>
Index: 1386 entries, ACADEMY DINOSAUR to ZORRO ARK
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1386 non-null   int64  
 1   description           1386 non-null   object 
 2   release_year          1386 non-null   int64  
 3   language_id           1386 non-null   int64  
 4   original_language_id  0 non-null      float64
 5   rental_duration       1386 non-null   int64  
 6   rental_rate           1386 non-null   float64
 7   length                1386 non-null   int64  
 8   replacement_cost      1386 non-null   float64
 9   rating                1386 non-null   object 
 10  special_features      1386 non-null   object 
 11  last_update           1386 non-null   object 
 12  category_id           1000 non-null   float64
dtypes: float64(4), int64(5), object(4)
memory usage: 151.6+ KB


In [66]:
# Le elimino duplicado que se generan

joined.drop_duplicates(inplace=True)

In [67]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, ACADEMY DINOSAUR to ZORRO ARK
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1000 non-null   int64  
 1   description           1000 non-null   object 
 2   release_year          1000 non-null   int64  
 3   language_id           1000 non-null   int64  
 4   original_language_id  0 non-null      float64
 5   rental_duration       1000 non-null   int64  
 6   rental_rate           1000 non-null   float64
 7   length                1000 non-null   int64  
 8   replacement_cost      1000 non-null   float64
 9   rating                1000 non-null   object 
 10  special_features      1000 non-null   object 
 11  last_update           1000 non-null   object 
 12  category_id           614 non-null    float64
dtypes: float64(4), int64(5), object(4)
memory usage: 109.4+ KB


In [68]:
# Le reseteo el indice

joined.reset_index(inplace=True)

In [69]:
joined

,title,film_id,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,category_id
0,ACADEMY DINOSAUR,1,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6.0
1,ACE GOLDFINGER,2,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42,11.0
2,ADAPTATION HOLES,3,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42,6.0
3,AFFAIR PREJUDICE,4,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42,NaN
4,AFRICAN EGG,5,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42,NaN
5,AGENT TRUMAN,6,A Intrepid Panorama of a Robot And a Boy who m...,2006,1,NaN,3,2.99,169,17.99,PG,Deleted Scenes,2006-02-15 05:03:42,9.0
6,AIRPLANE SIERRA,7,A Touching Saga of a Hunter And a Butler who m...,2006,1,NaN,6,4.99,62,28.99,PG-13,"Trailers,Deleted Scenes",2006-02-15 05:03:42,NaN
7,AIRPORT POLLOCK,8,A Epic Tale of a Moose And a Girl who must Con...,2006,1,NaN,6,4.99,54,15.99,R,Trailers,2006-02-15 05:03:42,NaN
8,ALABAMA DEVIL,9,A Thoughtful Panorama of a Database Administra...,2006,1,NaN,3,2.99,114,21.99,PG-13,"Trailers,Deleted Scenes",2006-02-15 05:03:42,11.0
9,ALADDIN CALENDAR,10,A Action-Packed Tale of a Man And a Lumberjack...,2006,1,NaN,6,4.99,63,24.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42,15.0
